# 1. Install dependencies

In [ ]:
from google.colab import output

# Update colab apt
!sudo apt update

# Install Aria2 for download torrents
!sudo apt install aria2 xattr

# Install python packages
!pip install requests

output.clear()
print('Success: Requirements installed!')

# 2. Configure download

In [ ]:
#@title 2.1 Initialize Google Drive

from google.colab import drive, output
drive.mount('/content/gdrive')

output.clear()
print('Success: Google Drive initialized')

In [ ]:
from google.colab import files

#@title 2.2 Initialize download parameters

torrent_url = ''  #@param {type: "string"}
proxy = ''  #@param {type: "string"}

gdrive_output = "Colab-Torrent" #@param {type:"string"}
drive_type = "My Drive" #@param ["My Drive", "Shared Drive"]
sharded_drive_name = '' #@param {type:"string"}

if len(torrent_url) < 1:
    files.upload_file('to_download.torrent')
    torrent_url = '/content/to_download.torrent'


# 3. Download content

In [ ]:
import subprocess
import time
import os
import shutil
import requests

!sudo rm -rf /temp/*-temp

temp_path = os.path.join('/temp', '%s-temp' % int(time.time()))
os.makedirs(temp_path, exist_ok=True)

if len(proxy) < 1:
  proxy_url = None
elif not proxy.startswith('http:'):
  raise Exception('Error: Only http proxy can be used!')

if drive_type == 'Shared Drive':
  if len(sharded_drive_name) < 1:
    raise Exception('Error: No shared drive name supplied')

  output_path = os.path.join('/content/gdrive/Shareddrives/', sharded_drive_name)

  if not os.path.exists(output_path):
    raise Exception('Error: Shared drive "%s" not found' % sharded_drive_name)
else:
  output_path = '/content/gdrive/MyDrive'

output_path = os.path.join(output_path, gdrive_output)
os.makedirs(output_path, exist_ok=True)

if torrent_url.startswith('http'):
  torrent_path = os.path.join(temp_path, 'torrent-%s.torrent' % int(time.time()))

  with open(torrent_path, 'wb') as fp:
    req = requests.get(torrent_url)

    if req.status_code != 200:
      raise Exception('Error: Unable to get torrent')
    fp.write(req.content)

  torrent_url = torrent_path

args = [
  'aria2c',
  '-T', torrent_url,
  '-d', temp_path,
  '--remote-time',
  '-x', '16',
  '-j', '16',
  '-s', '16',
  '--min-split-size', '20M',
  '--allow-overwrite=true',
  '--auto-file-renaming=false',
  '--retry-wait', '2',
  '--max-tries', '5',
  '--max-file-not-found', '0',
  '--summary-interval', '0',
  '--seed-time=0',
  '--summary-interval=1'
]

if proxy:
  args += ["--all-proxy", str(proxy)]

proc = subprocess.Popen(
    args,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.PIPE
)

print('Info: Downloading content ...')
_, err = proc.communicate()

if proc.returncode != 0:
  raise Exception('Error: \n' + err.decode('utf-8'))

if os.path.exists(torrent_url):
  os.unlink(torrent_url)

try:
  for item in os.scandir(temp_path):
    print('Info: Upload %s' % os.path.basename(item.path))

    if os.path.exists(os.path.join(output_path, os.path.basename(item.path))):
      if item.is_file:
        os.unlink(os.path.join(output_path, os.path.basename(item.path)))
      else:
        shutil.rmtree(os.path.join(output_path, os.path.basename(item.path)))

    shutil.move(item.path, output_path)
  drive.flush_and_unmount()
except:
  raise Exception('Error: Unable to upload files to Google Drive')

print('Success: File downloaded!')
